In [ ]:
%pip install google-auth-oauthlib
%pip install --upgrade google-api-python-client

In [4]:
%pip install aiofiles

Note: you may need to restart the kernel to use updated packages.


In [1]:
import aiohttp
import aiofiles
import json
from google.oauth2.service_account import Credentials
from datetime import datetime, timedelta

async def get_views(page_path: str):
    SCOPES = ['https://www.googleapis.com/auth/analytics.readonly']
    
    async with aiofiles.open('/home/anon-b/Downloads/toknowviews-1730747751593-d5a762fdba11.json', mode='r') as f:
        service_account_info = json.loads(await f.read())
    
    credentials = Credentials.from_service_account_info(service_account_info, scopes=SCOPES)
    token = credentials.token
    
    tomorrow_date = (datetime.now() + timedelta(days=1)).strftime('%Y-%m-%d')
    body = {
        "dateRanges": [{
            "startDate": "2024-01-01",
            "endDate": tomorrow_date
        }],
        "dimensions": [{
            "name": "pagePath"
        }],
        "dimensionFilter": {
            "filter": {
                "fieldName": "pagePath",
                "stringFilter": {
                    "matchType": "CONTAINS",
                    "value": page_path
                }
            }
        },
        "metrics": [{
            "name": "screenPageViews"
        }]
    }
    
    headers = {
        'Authorization': f'Bearer {token}',
        'Content-Type': 'application/json'
    }
    
    async with aiohttp.ClientSession() as session:
        async with session.post('https://analyticsdata.googleapis.com/v1beta/properties/447265843:runReport', headers=headers, json=body) as response:
            response_json = await response.json()
    
    counts = [
        int(metricValue.get('value', 0))
        for row 
        in response_json.get('rows', []) 
            for metricKey, metricValues 
            in row.items() 
                for metricValue 
                in metricValues
                if metricKey == 'metricValues'
    ]
    return sum(counts)

In [5]:
await get_views("/about")

0

<https://gemini.google.com/app/9a1a99a1df31aab5>

<https://claude.ai/chat/4309b5b3-dae7-44d1-ab94-4e3cd4a21633>

<https://claude.ai/chat/30125116-bf87-42b5-9d71-09962e619a7e>

<https://developers.cloudflare.com/workers/languages/python/packages/#supported-packages>

In [27]:
from google.oauth2.service_account import Credentials
from googleapiclient.discovery import build
from datetime import datetime, timedelta

# get_views("/posts/private-domain-checker")
# get_views("/about")
# get_views("/posts/artificial-general-intelligence")
def get_views(page_path: str):
    SCOPES = ['https://www.googleapis.com/auth/analytics.readonly']
    credentials = Credentials.from_service_account_file(
        '/home/anon-b/Downloads/toknowviews-1730747751593-d5a762fdba11.json', 
        scopes=SCOPES)
    analytics = build('analyticsdata', 'v1beta', credentials = credentials)
    tomorrow_date = (datetime.now() + timedelta(days=1)).strftime('%Y-%m-%d')
    body = {
        "dateRanges": [{
            "startDate": "2024-01-01",
            "endDate": tomorrow_date
        }],
        "dimensions": [{
            "name": "pagePath"
        }],
        "dimensionFilter": {
            "filter": {
                "fieldName": "pagePath",
                "stringFilter": {
                    "matchType": "CONTAINS",
                    "value": page_path
                }
            }
        },
        "metrics": [{
            "name": "screenPageViews"
        }]
    }
    response = analytics.properties().runReport(
        property="properties/447265843",
        body=body).execute()
    counts = [
        int(metricValue.get('value', 0))
        for row 
        in response.get('rows', []) 
            for metricKey,metricValues 
            in row.items() 
                for metricValue 
                in metricValues
                if metricKey == 'metricValues'
    ]
    return sum(counts)

# New helper function to generate SVG badge
def generate_views_badge(views: int):
  views_text = str(views)
  text_width = len(views_text) * 8 + 10 # Approximate width calculation
  total_width = 80 + text_width  # Label + padding + views width
  
  return f'''<?xml version="1.0" encoding="UTF-8"?>
    <svg xmlns="http://www.w3.org/2000/svg" width="{total_width}" height="20">
      <linearGradient id="b" x2="0" y2="100%">
        <stop offset="0" stop-color="#bbb" stop-opacity=".1"/>
        <stop offset="1" stop-opacity=".1"/>
      </linearGradient>
      <mask id="a">
        <rect width="{total_width}" height="20" rx="3" fill="#fff"/>
      </mask>
      <g mask="url(#a)">
        <rect width="80" height="20" fill="#555"/>
        <rect x="80" width="{text_width}" height="20" fill="#4c1"/>
        <rect width="{total_width}" height="20" fill="url(#b)"/>
      </g>
      <g fill="#fff" text-anchor="middle" font-family="DejaVu Sans,Verdana,Geneva,sans-serif" font-size="11">
        <text x="40" y="15" fill="#010101" fill-opacity=".3">Page Views</text>
        <text x="40" y="14">Page Views</text>
        <text x="{80 + text_width/2}" y="15" fill="#010101" fill-opacity=".3">{views_text}</text>
        <text x="{80 + text_width/2}" y="14">{views_text}</text>
      </g>
    </svg>'''.strip()

def format_number(num: int|float):
    """
    Format a number similar to YouTube's style:
    - Less than 1000: show as is (100, 999)
    - Thousands: show as K (1K, 1.3K)
    - Millions: show as M (1.1M, 2.4M)
    
    Args:
        num (int/float): Number to format
        
    Returns:
        str: Formatted number string
    """
    abs_num = abs(num)
    sign = '-' if num < 0 else ''
    
    if abs_num < 1000:
        return f"{sign}{abs_num:d}"
        
    elif abs_num < 1000000:
        formatted = abs_num / 1000
        # If the decimal part is 0, don't show it
        if formatted.is_integer():
            return f"{sign}{int(formatted)}K"
        return f"{sign}{formatted:.1f}K"
        
    else:
        formatted = abs_num / 1000000
        if formatted.is_integer():
            return f"{sign}{int(formatted)}M"
        return f"{sign}{formatted:.1f}M"

In [28]:
from IPython.display import SVG, display

svg_code = generate_views_badge(format_number(2235433))
display(SVG(svg_code))